In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset
train_dataset = load_dataset("LabHC/bias_in_bios", split='train')
test_dataset = load_dataset("LabHC/bias_in_bios", split='test')
dev_dataset = load_dataset("LabHC/bias_in_bios", split='dev')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/257478 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/99069 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/39642 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

# Convert the dataset to a pandas DataFrame
train_df = pd.DataFrame(train_dataset)
test_df = pd.DataFrame(test_dataset)
dev_df = pd.DataFrame(dev_dataset)

# View the DataFrame
print(train_df.head())

                                           hard_text  profession  gender
0  He is also the project lead of and major contr...          21       0
1  She is able to assess, diagnose and treat mino...          13       1
2  Prior to law school, Brittni graduated magna c...           2       1
3  He regularly contributes to India’s First Onli...          11       0
4  He completed his medical degree at Northwester...          21       0


In [ ]:
import pandas as pd

# Combine the three DataFrames into one DataFrame
#combined_df = pd.concat([train_df, test_df, dev_df], axis=0)
#combined_df = pd.concat([test_df, dev_df], axis=0)
combined_df = pd.concat([dev_df], axis=0)

# Reset the index of the new combined DataFrame
combined_df.reset_index(drop=True, inplace=True)

# View the combined DataFrame
print(combined_df.head())

                                           hard_text  profession  gender
0  Tampio researches the history of political tho...          21       0
1  He has been a practicing Dentist for 11 years....           6       0
2  Mr. Miserez devotes a substantial portion of h...           2       0
3  She holds a master’s degree in nutrition educa...           7       1
4  She received her Ph.D. in 2013 from Florida In...          21       1


In [ ]:
combined_df = combined_df.iloc[:10000,:]
combined_df

,hard_text,profession,gender
0,Tampio researches the history of political tho...,21,0
1,He has been a practicing Dentist for 11 years....,6,0
2,Mr. Miserez devotes a substantial portion of h...,2,0
3,She holds a master’s degree in nutrition educa...,7,1
4,She received her Ph.D. in 2013 from Florida In...,21,1
...,...,...,...
9995,He grew up in an artist household where his pa...,9,0
9996,He has done a study on the death of Jesus and ...,21,0
9997,He is passionate about building up DC's queer ...,8,0
9998,In the last years she is enjoying part-time mo...,12,1


In [ ]:
# Combine all hard_text entries into a single string with <EOS> at the end of each entry
text = ' <EOS> '.join(combined_df['hard_text']) + ' <EOS> '

In [ ]:
# This is our corpus for training Skip-gram Word2Vec
print(text[-500:])
print(len(text))

building her portfolio, her experience and shaping her modeling style. While living in Reddingshe has also shot all over California and Nevada. She is known for her unique sense of fashion and versatility of style. While she is kinda shy a.. 5′7ʺ / 170cm Born 1996 <EOS> Mr. Tartarilla practices medicine in Fort Walton Beach, FL and specializes in Family Medicine. Mr. Tartarilla is affiliated with Fort Walton Beach Medical Center. Mr. Tartarilla speaks English, French, Chinese and Spanish. <EOS> 
4027989


In [ ]:
text[0:200]

'Tampio researches the history of political thought, contemporary political theory, and education policy. His first book, Kantian Courage, explored how Anglo-American, Continental, and Islamic politica'

In [ ]:
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchtext.data.utils import get_tokenizer
from torchtext.datasets import DATASETS
from torchtext.vocab import build_vocab_from_iterator
import torch.nn as nn
from tqdm import tqdm
import pickle
import random
import numpy as np
from collections import Counter, defaultdict
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
import gensim.downloader
from torch import FloatTensor as FT

In [ ]:
# Where do I want to run my job. You can do "cuda" on linux machines.
DEVICE = "cuda" if torch.cuda.is_available() else  "cpu"
# The batch size in Adam or SGD.
BATCH_SIZE = 256
# Number of epochs.
NUM_EPOCHS = 5
# Predict from 2 words the inner word for CBOW.
# I.e. I'll have a window like ["a", "b", "c"] of continuous text (each is a word).
# We'll predict each of wc = ["a", "c"] from "b" = wc for Skip-Gram.
# For CBOW, we'll use ["a", "c"] to predict "b" = wo.

# This is *NOT* used. This is "m" in lecture. It is random for each center word. See below.
WINDOW = 1

# Negative samples.
K = 4

In [ ]:
print(DEVICE)

cuda


In [ ]:
punc = '!"#$%&()*+,-./:;=?@[\\]^_\'{|}~\t\n'

# Can do regular expressions here too.
for c in punc:
    if c in text:
        text = text.replace(c, '')

In [ ]:
text[0:200]

'Tampio researches the history of political thought contemporary political theory and education policy His first book Kantian Courage explored how AngloAmerican Continental and Islamic political theori'

In [ ]:
# A very crude tokenizer you get for free: lower case and also split on spaces.
TOKENIZER = get_tokenizer("basic_english")

In [ ]:
words = TOKENIZER(text)
f = Counter(words)

In [ ]:
len(words)

614318

In [ ]:
# Do a very crude filter on the text
text = [word for word in words if f[word] > 2]
text = [word if word != '<eos>' else '<EOS>' for word in text]

In [ ]:
text[0:10]

['researches',
 'the',
 'history',
 'of',
 'political',
 'thought',
 'contemporary',
 'political',
 'theory',
 'and']

In [ ]:
VOCAB = build_vocab_from_iterator([text], specials=["<EOS>"])

In [ ]:
len(VOCAB)

12448

In [ ]:
# word -> int hash map.
stoi = VOCAB.get_stoi()
# int -> word hash map.
itos = VOCAB.get_itos()

In [ ]:
stoi['<EOS>']

0

In [ ]:
stoi['as']

18

In [ ]:
# Total number of words.
len(stoi)

12448

In [ ]:
f = Counter(text)
# This is the probability that we pick a word in the corpus.
z = {word: f[word] / len(text) for word in f}

In [ ]:
threshold = 1e-5
# Probability that word is kept while subsampling.
# This is explained here and sightly differet from the paper: http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/
p_keep = {word: (np.sqrt(z[word] / 0.001) + 1)*(0.0001 / z[word]) for word in f}

In [ ]:
# This is in the integer space.
train_dataset = [word for word in text if random.random() < p_keep[word]]

# Rebuild the vocabulary.
VOCAB = build_vocab_from_iterator([train_dataset], specials=["<EOS>"])

In [ ]:
len(train_dataset)

227305

In [ ]:
# word -> int mapping.
stoi = VOCAB.get_stoi()
# int -> word mapping.
itos = VOCAB.get_itos()

In [ ]:
stoi['<EOS>']

0

In [ ]:
# The vocabulary size after we do all the filters.
len(VOCAB)

12448

In [ ]:
# The probability we draw something for negative sampling.
f = Counter(train_dataset)
p = torch.zeros(len(VOCAB))

# Downsample frequent words and upsample less frequent.
s = sum([np.power(freq, 0.75) for word, freq in f.items()])

for word in f:
    p[stoi[word]] = np.power(f[word], 0.75) / s

In [ ]:
# Map everything to integers.
train_dataset = [stoi[word] for word in text]

In [ ]:
# This just gets the (wc, wo) pairs that are positive - they are seen together!
def get_tokenized_dataset(dataset, verbose=False):
    x_list = []
    sentence = []  # Temporary list to hold words of the current sentence

    for i, token in enumerate(dataset):
        if token == stoi['<EOS>'] or i == len(dataset) - 1:  # End of sentence or end of dataset
            # Process the accumulated sentence
            for j, wc in enumerate(sentence):
                # Pick a random window size for each center word
                m = random.randint(1, 5)

                # Calculate start and end indices for context words within the sentence
                start = max(0, j - m)
                end = min(j + m + 1, len(sentence))

                # Gather context words, excluding the center word itself
                target_tokens = sentence[start:j] + sentence[j + 1:end]
                if verbose:
                  print(m)
                  print(target_tokens)
                # Extend x_list with (center word, context word) pairs
                for wo in target_tokens:
                    x_list.append([wc, wo])
                    if verbose:
                        print(f"[{wc}, {wo}]")

            # Reset the sentence list after processing
            sentence = []
        else:
            # Accumulate words of the current sentence
            sentence.append(token)

    return x_list


In [ ]:
def get_tokenized_dataset(dataset, verbose=False):
    x_list = []
    sentence = []  # Temporary list to hold words (indices) of the current sentence

    for i, token in enumerate(dataset):  # Iterate through the dataset
        if token == stoi['<EOS>']:  # Check for end of sentence marker using its index
            if verbose:
                # When printing, you might want to convert indices back to words using `itos` for readability
                readable_sentence = [itos[token] for token in sentence]
                print(f"Current sentence: {readable_sentence}")
            # Process the accumulated sentence
            for j, wc in enumerate(sentence):
                # Pick a random window size for each center word
                #m = random.randint(1, int(len(sentence)/2))
                m = int(len(sentence)/2)
                # Calculate start and end indices for context words within the sentence
                start = max(0, j - m)
                end = min(j + m + 1, len(sentence))

                # Gather context words, excluding the center word itself
                target_tokens = sentence[start:j] + sentence[j + 1:end]

                if verbose:
                    readable_context = [itos[token] for token in target_tokens]
                    print(f"Center word: {itos[wc]}, Context: {readable_context}")

                # Extend x_list with (center word, context word) pairs
                for wo in target_tokens:
                    x_list.append([wc, wo])

            # Reset the sentence list after processing
            sentence = []
        else:
            # Accumulate words (indices) of the current sentence
            sentence.append(token)

    return x_list

In [ ]:
train_x_list = get_tokenized_dataset(train_dataset, verbose=False)

In [ ]:
text[-20:]

['medicine',
 'mr',
 'tartarilla',
 'is',
 'affiliated',
 'with',
 'fort',
 'walton',
 'beach',
 'medical',
 'center',
 'mr',
 'tartarilla',
 'speaks',
 'english',
 'french',
 'chinese',
 'and',
 'spanish',
 '<EOS>']

In [ ]:
pickle.dump(train_x_list, open('train_x_list.pkl', 'wb'))

In [ ]:
train_x_list = pickle.load(open('train_x_list.pkl', 'rb'))

In [ ]:
# These are (wc, wo) pairs. All are y = +1 by design.
train_x_list[-10:]

[[241, 1423],
 [241, 21],
 [241, 59],
 [241, 188],
 [241, 12133],
 [241, 145],
 [241, 116],
 [241, 792],
 [241, 615],
 [241, 1]]

In [ ]:
# The number of things of BATCH_SIZE = 512.
len(train_x_list) // BATCH_SIZE

114588

### Set up the dataloader.

In [ ]:
train_dl = DataLoader(
    TensorDataset(
        torch.tensor(train_x_list).to(DEVICE),
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)

### Words we'll use to asses the quality of the model ...

In [ ]:
valid_ids = torch.tensor([
    stoi['women'],
    stoi['men'],
    stoi['nurse'],
    stoi['doctor'],
    stoi['engineer'],
    stoi['teacher'],
    stoi['dietitian'],
])

### Get the model
- See docs for nn.Embedding: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html
- The same: nn.Embedding (2) = nn.Embedding.weight * oneHot(2)

In [ ]:
class SkipGramNegativeSampling(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(SkipGramNegativeSampling, self).__init__()
        self.A = nn.Embedding(vocab_size, embed_dim) # Context vectors - center word.
        self.B = nn.Embedding(vocab_size, embed_dim) # Output vectors - words around the center word.
        self.init_weights()

    def init_weights(self):
        # Is this the best way? Not sure.
        initrange = 0.5
        self.A.weight.data.uniform_(-initrange, initrange)
        self.B.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        # N is the batch size.
        # x is (N, 2)

        # Each of these is (N, 1)
        wc, wo = x[:, 0], x[:, 1]

        # Each of these is (N, 1, D) since each context has 1 word.
        a = self.A(wc)

        # Each of these is (N, 1, D) since each target has 1 word.
        b = self.B(wo)

        # The product between each context and target vector.
        # Each of these is (N, 1, D) since each batch has 1 word.
        # The logits is now (N, 1) since we sum across the final dimension.
        logits = (a * b).sum(axis=-1)

        return logits

In [ ]:
@torch.no_grad()
def validate_embeddings(
    model,
    valid_ids,
    itos
):
    """ Validation logic """

    # We will use context embeddings to get the most similar words
    # Other strategies include: using target embeddings, mean embeddings after avaraging context/target
    embedding_weights = model.A.weight

    normalized_embeddings = embedding_weights.cpu() / np.sqrt(
        np.sum(embedding_weights.cpu().numpy()**2, axis=1, keepdims=True)
    )

    # Get the embeddings corresponding to valid_term_ids
    valid_embeddings = normalized_embeddings[valid_ids, :]

    # Compute the similarity between valid_term_ids (S) and all the embeddings (V).
    # We do S x d (d x V) => S x D and sort by negative similarity.
    top_k = 10 # Top k items will be displayed.
    similarity = np.dot(valid_embeddings.cpu().numpy(), normalized_embeddings.cpu().numpy().T)

    # Invert similarity matrix to negative
    # Ignore the first one because that would be the same word as the probe word
    similarity_top_k = np.argsort(-similarity, axis=1)[:, 1: top_k+1]

    # Print the output.
    for i, word_id in enumerate(valid_ids):
        # j >= 1 here since we don't want to include the word itself.
        similar_word_str = ', '.join([itos[j] for j in similarity_top_k[i, :] if j >= 1])
        print(f"{itos[word_id]}: {similar_word_str}")

    print('\n')

### Set up the model

In [ ]:
LR = 10.0
NUM_EPOCHS = 5
EMBED_DIM = 300

In [ ]:
model = SkipGramNegativeSampling(len(VOCAB), EMBED_DIM).to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
# The learning rate is lowered every epoch by 1/10.
# Is this a good idea?
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [ ]:
model

SkipGramNegativeSampling(
  (A): Embedding(12448, 300)
  (B): Embedding(12448, 300)
)

In [ ]:
validate_embeddings(model, valid_ids, itos)

women: 2011, christina, bentham, happiness, starting, men’s, mines, photojournalism, twenty, utmb
men: mimi, longrun, arriving, editorinchief, asbestos, usa, sell, transplant, blanchard, strand
nurse: malaria, vietnam, finest, generations, ebook, ocular, kinsey, random, melissa, vessels
doctor: dietitian, maori, livelihoods, smart, neglected, request, bartosz, wichita, gerontology, crystals
engineer: curry, retire, crisp, practiced, mediums, soughtafter, wrongful, acclaim, 500, locations
teacher: getty, ”, 37, genealogy, t, ramon, baltic, extended, summa, unconventional
dietitian: doctor, flute, intersections, anthem, montclair, doctoral, municipalities, alfaro, ephemera, donna




### Train the model

In [ ]:
def train(dataloader, model, optimizer, epoch):
    model.train()
    total_acc, total_count, total_loss, total_batches = 0, 0, 0.0, 0.0
    log_interval = 500

    for idx, x_batch in tqdm(enumerate(dataloader)):

        x_batch = x_batch[0]

        batch_size = x_batch.shape[0]

        optimizer.zero_grad()

        logits = model(x_batch)

        # Get the positive samples loss. Notice we use weights here
        positive_loss = torch.nn.BCEWithLogitsLoss()(input=logits, target=torch.ones(batch_size).to(DEVICE).float())

        # For each batch, get some negative samples
        # We need a total of B * 2 * WINDOW * K samples across a batch
        # We then reshape this batch
        # These are effectively the output words
        negative_samples = torch.multinomial(p, batch_size * K, replacement=True)

        # Repeat the center word enough times so we can merge as needed
        # Repeat does not work on MPS apparently
        wc = x_batch[:, 0].to("cpu").repeat(K).sort().values
        wo = negative_samples

        # Get the negative samples
        x_batch_negative = torch.stack([wc, wo], axis=1).to(DEVICE)

        """
        Note the way we formulated the targets: they are all 0 since these are negative samples.
        We do the BCEWithLogitsLoss by hand basically here.
        Notice we sum across the negative samples, per positive word.

        This is literally the equation in the lecture notes.
        """
        negative_loss = model(x_batch_negative).neg().sigmoid().log().reshape(
            batch_size, K
        ).sum(1).mean().neg().to(DEVICE)

        loss = (positive_loss + negative_loss).mean()

        # Get the gradients via back propagation
        loss.backward()

        # Clip the gradients? Generally a good idea
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)

        # Do an optimization step. Update the parameters A and B
        optimizer.step()
        total_loss += loss.item()
        total_batches += 1

        if idx % log_interval == 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| loss {:8.3f} ".format(
                    epoch,
                    idx,
                    len(dataloader),
                    total_loss / total_batches
                )
            )
            validate_embeddings(model, valid_ids, itos)
            total_loss, total_batches = 0.0, 0.0

### Some results from the run look like below:

In [ ]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()

    train(train_dl, model, optimizer, epoch)
    # We have a learning rate scheduler here
    # Basically, given the state of the optimizer, this lowers the learning rate in a smart way
    scheduler.step()

1it [00:04,  4.08s/it]

| epoch   1 |     0/114589 batches | loss    4.534 
women: 2011, christina, bentham, happiness, starting, men’s, mines, photojournalism, twenty, utmb
men: mimi, longrun, arriving, editorinchief, asbestos, usa, sell, transplant, blanchard, strand
nurse: malaria, vietnam, finest, generations, ebook, ocular, kinsey, random, melissa, vessels
doctor: dietitian, maori, livelihoods, smart, neglected, request, bartosz, wichita, gerontology, crystals
engineer: curry, retire, crisp, practiced, mediums, soughtafter, wrongful, acclaim, 500, locations
teacher: getty, ”, 37, genealogy, t, ramon, baltic, extended, summa, unconventional
dietitian: doctor, flute, intersections, anthem, montclair, doctoral, municipalities, alfaro, ephemera, donna




532it [00:06, 229.63it/s]

| epoch   1 |   500/114589 batches | loss    4.138 
women: 2011, christina, and, to, bentham, starting, center, effort, happiness, men’s
men: mimi, arriving, longrun, editorinchief, asbestos, usa, sell, strand, blanchard, transplant
nurse: malaria, vietnam, generations, ebook, finest, ocular, melissa, to, random, kinsey
doctor: dietitian, maori, smart, livelihoods, neglected, request, bartosz, wichita, sexuality, governor
engineer: retire, curry, practiced, crisp, soughtafter, mediums, acclaim, locations, wrongful, music
teacher: getty, ”, summa, extended, genealogy, 37, baltic, business, unconventional, seventeenth
dietitian: doctor, flute, intersections, anthem, montclair, doctoral, municipalities, ephemera, donna, alfaro




1026it [00:08, 225.94it/s]

...

114033it [07:44, 230.67it/s]

| epoch   5 | 114000/114589 batches | loss    1.633 
women: women’s, the, she, about, gender, her, female, through, most, men
men: women, to, because, one, says, about, life, the, much, who
nurse: practitioner, affiliates, cooperates, experiences, diverse, specialists, nursing, having, especially, psychologist
doctor: provided, dental, tooth, some, impacted, association, member, fixing, implant, college
engineer: engineering, software, technical, systems, computer, computing, worked, electrical, senior, company
teacher: teachers, education, curriculum, elementary, teaching, classroom, instruction, schools, students, taught
dietitian: nutrition, with, having, experiences, diverse, practitioner, specialists, groups, affiliates, cooperates




114529it [07:45, 223.78it/s]

| epoch   5 | 114500/114589 batches | loss    1.634 
women: women’s, the, she, about, gender, her, female, through, most, men
men: women, to, because, one, says, about, life, the, much, who
nurse: practitioner, affiliates, cooperates, experiences, diverse, specialists, nursing, having, especially, psychologist
doctor: provided, dental, tooth, some, impacted, association, member, fixing, implant, college
engineer: engineering, software, technical, systems, computer, computing, worked, electrical, senior, company
teacher: teachers, education, curriculum, elementary, teaching, classroom, instruction, schools, students, taught
dietitian: nutrition, with, having, experiences, diverse, practitioner, specialists, groups, affiliates, cooperates




114589it [07:49, 244.00it/s]


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'nurse'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between nurse and average female-related words: 0.40602073
Similarity between nurse and average male-related words: 0.3403736


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'doctor'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between doctor and average female-related words: 0.37957415
Similarity between doctor and average male-related words: 0.41495195


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'teacher'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between teacher and average female-related words: 0.4053283
Similarity between teacher and average male-related words: 0.41434962


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'professor'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between professor and average female-related words: 0.2916693
Similarity between professor and average male-related words: 0.36610782


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'engineer'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between engineer and average female-related words: 0.24346823
Similarity between engineer and average male-related words: 0.37673298


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'journalist'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between journalist and average female-related words: 0.46276832
Similarity between journalist and average male-related words: 0.40792114


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'dietitian'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between dietitian and average female-related words: 0.33664325
Similarity between dietitian and average male-related words: 0.2700533


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'dentist'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between dentist and average female-related words: 0.30511302
Similarity between dentist and average male-related words: 0.32771423
